In [ ]:
### SBB.py program

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql import HiveContext, SQLContext,SparkSession
from pyspark.sql import functions as F
from datetime import date,timedelta
from datetime import datetime
sc = SparkContext(appName = "Combined_Table")
sql_context = HiveContext(sc)
spark = SparkSession(sc)
spark = (SparkSession.builder.enableHiveSupport().getOrCreate())


In [ ]:
spark 
import pandas as pd
import os
import numpy as np
import datetime
import time
import glob
import os
from numpy import nan
from pyspark.sql import functions as F
from pyspark.sql.functions import format_string

pd.set_option('display.width', 150)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
exec(open('/home/sharb24/SBB/thural2/record.py').read())


In [ ]:
rec_date

In [ ]:
####################### FIRST PROCESS THE GROSS VOLUME SEGMENTATION METRICS###########################################
cred_fil=spark.sql("select * from anp_cabbtdct1_final.psa_elig_retail")
cred_ret_filter=cred_fil.toPandas()

## change psa_month to month name
cred_ret_filter=cred_ret_filter.astype({"psa_month": int})
import calendar
cred_ret_filter['psa_month'] = cred_ret_filter['psa_month'].apply(lambda x: calendar.month_name[x]).str.lower()
#cred_ret_filter.head(10)

In [ ]:
#### subset for GROSS VOLUME SEGMENTATION METRICS AND REST OF PSA METRICS
cred_ret_filter0=cred_ret_filter[(cred_ret_filter.seg_ind!=0)]
cred_ret_filter00=filterfunc1(cred_ret_filter0)

## create metric name for rest of psa metrics

############################# GROSS VOLUME ###########################################
## SBB TAB: TOTAL SBB GROSS VOLUME
sbb_gross_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(gross_all_tuple)]
sbb_gross_filter_ret=sbb_gross_filter_ret.assign(metric_name='Total SBB Gross Volume')
sbb_gross_filter_ret=filterfunc1(sbb_gross_filter_ret)

## SBB GROSS LOC VOLUME
gross_loc_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(gross_loc_tuple)]
gross_loc_filter_ret=gross_loc_filter_ret.assign(metric_name='SBB Gross LOC Volume')
gross_loc_filter_ret=filterfunc1(gross_loc_filter_ret)

## SBB GROSS LON VOLUME
gross_lon_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(gross_lon_tuple)]
gross_lon_filter_ret=gross_lon_filter_ret.assign(metric_name='SBB Gross LON Volume')
gross_lon_filter_ret=filterfunc1(gross_lon_filter_ret)

## SBB GROSS PROFESSIONAL STUDENT LOC VOLUME
gross_prof_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(gross_student_loc_tuple)]
gross_prof_filter_ret=gross_prof_filter_ret.assign(metric_name='Gross Professional Student LOC')
gross_prof_filter_ret=filterfunc1(gross_prof_filter_ret)

### APPEND ALL THE DATAFRAME FILTERS
gross_all_ret=pd.concat([sbb_gross_filter_ret,gross_loc_filter_ret,gross_lon_filter_ret,gross_prof_filter_ret],axis=0)

########################## CREDIT CARDS ##########################################

## SBB TAB: TOTAL CREDIT CARDS
sbb_cc_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(total_cc_tuple)]
sbb_cc_filter_ret=sbb_cc_filter_ret.assign(metric_name='Total Credit Cards')
sbb_cc_filter_ret=filterfunc1(sbb_cc_filter_ret)

## SBB TAB: BUSINESS VISA
sbb_bv_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(business_visa_cc_tuple)]
sbb_bv_filter_ret=sbb_bv_filter_ret.assign(metric_name='Credit Cards - Business VISA')
sbb_bv_filter_ret=filterfunc1(sbb_bv_filter_ret)

## SBB TAB: BUSINESS VISA
sbb_btv_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(business_travel_tuple)]
sbb_btv_filter_ret=sbb_btv_filter_ret.assign(metric_name='Credit Cards - Business Travel VISA')
sbb_btv_filter_ret=filterfunc1(sbb_btv_filter_ret)

## SBB TAB: AEROPLAN BUSINESS
sbb_aero_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(business_aero_tuple)]
sbb_aero_filter_ret=sbb_aero_filter_ret.assign(metric_name='Credit Cards - Aeroplan Business')
sbb_aero_filter_ret=filterfunc1(sbb_aero_filter_ret)

## SBB TAB: BUSINESS SELECT NO FEE
sbb_nofee_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(bus_nofee_tuple)]
sbb_nofee_filter_ret=sbb_nofee_filter_ret.assign(metric_name='Credit Cards - Business Select No Fee')
sbb_nofee_filter_ret=filterfunc1(sbb_nofee_filter_ret)

## SBB TAB: BUSINESS SELECT FEE
sbb_fee_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(bus_fee_tuple)]
sbb_fee_filter_ret=sbb_fee_filter_ret.assign(metric_name='Credit Cards - Business Select Fee')
sbb_fee_filter_ret=filterfunc1(sbb_fee_filter_ret)

## SBB TAB: CREDIT CARD INCREASES (CCI) - New product added
sbb_cci_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(business_essentials_credit_card_increases)]
sbb_cci_filter_ret=sbb_cci_filter_ret.assign(metric_name='Credit Cards Increases')
sbb_cci_filter_ret=filterfunc1(sbb_cci_filter_ret)

### APPEND ALL THE DATAFRAME FILTERS
cc_all_ret=pd.concat([sbb_cc_filter_ret,sbb_bv_filter_ret,sbb_btv_filter_ret,sbb_aero_filter_ret,
                  sbb_nofee_filter_ret,sbb_fee_filter_ret, sbb_cci_filter_ret],axis=0)

########################## MERCHANT SOLUTIONS ##########################################

## SBB TAB: TOTAL MERCHANT SOLUTIONS
merchant_all_ret=cred_ret_filter[cred_ret_filter.product_code.isin(total_mer_tuple)]
merchant_all_ret=merchant_all_ret.assign(metric_name='Total Merchant Solutions')
merchant_all_ret=filterfunc1(merchant_all_ret)

################################ CMS ###############################################

## SBB TAB: TOTAL CMS
cms_all_ret=cred_ret_filter[cred_ret_filter.product_code.isin(total_cms_tuple)]
cms_all_ret=cms_all_ret.assign(metric_name='Total Cash Management Services (including RDC)')
cms_all_ret=filterfunc1(cms_all_ret)

################################ COMMERCIAL VOLUME ###################################

## SBB TAB: TOTAL COMMERCIAL VOLUME
sbb_com_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(total_com_tuple)]
sbb_com_filter_ret=sbb_com_filter_ret.assign(metric_name='ALL Total Commercial Volume')
sbb_com_filter_ret=filterfunc1(sbb_com_filter_ret)

## SBB TAB: COMMERCIAL DEPOSIT VOLUME
sbb_dep_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(com_deposit_tuple)]
sbb_dep_filter_ret=sbb_dep_filter_ret.assign(metric_name='All Commercial Deposit Volume')
sbb_dep_filter_ret=filterfunc1(sbb_dep_filter_ret)

## SBB TAB: COMMERCIAL LOAN VOLUME
sbb_loan_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(com_loan_tuple)]
sbb_loan_filter_ret=sbb_loan_filter_ret.assign(metric_name='All Commercial Loan Volume')
sbb_loan_filter_ret=filterfunc1(sbb_loan_filter_ret)

### APPEND ALL THE DATAFRAME FILTERS
com_all_ret=pd.concat([sbb_com_filter_ret,sbb_dep_filter_ret,sbb_loan_filter_ret],axis=0)

################################ NEW ACCOUNT OPENINGS ###############################

## SBB TAB: TOTAL ACCOUNT OPENINGS
sbb_open_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(total_new_tuple)]
sbb_open_filter_ret=sbb_open_filter_ret.assign(metric_name='Total New SBB Chequing Accounts')
sbb_open_filter_ret=filterfunc1(sbb_open_filter_ret)

## SBB TAB: GIC -- New product added
sbb_gic_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(gic_sbb)]
sbb_gic_filter_ret=sbb_gic_filter_ret.assign(metric_name='GICs')
sbb_gic_filter_ret=filterfunc1(sbb_gic_filter_ret)

################################ TD SECURITIES DIRECT TRADE ###########################

## SBB TAB: TOTAL TD SECURITIES DIRECT TRADE
sbb_trade_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(total_trade_tuple)]
sbb_trade_filter_ret=sbb_trade_filter_ret.assign(metric_name='TD Securities Direct Trade')
sbb_trade_filter_ret=filterfunc1(sbb_trade_filter_ret)

################################ BCP WIDGETS ##########################################

## SBB TAB: TOTAL BCP WIDGETS
sbb_bcp_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(total_bcp_tuple)]
sbb_bcp_filter_ret=sbb_bcp_filter_ret.assign(metric_name='Business Credit Protection(BCP) Widgets')
sbb_bcp_filter_ret=filterfunc1(sbb_bcp_filter_ret)

############################# MUR MORTGAGE VOLUME######################################

## SBB TAB: MUR MORTGAGE VOLUME
sbb_mur_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(total_mur_tuple)]
sbb_mur_filter_ret=sbb_mur_filter_ret.assign(metric_name='Total MUR Mortgage Volume')
sbb_mur_filter_ret=filterfunc1(sbb_mur_filter_ret)

################################ WEALTH REFERRAL WIDGETS #############################

## NO REATIL WEALTH REFERRALS

########################################### BODP ############################################

## SBB TAB: BODP
sbb_bodp_filter_ret=cred_ret_filter[cred_ret_filter.product_code.isin(total_bodp_tuple)]
sbb_bodp_filter_ret=sbb_bodp_filter_ret.assign(metric_name='Total BODP')
sbb_bodp_filter_ret=filterfunc1(sbb_bodp_filter_ret)


In [ ]:
################################# COUNT WIDGETS FOR AM ####################################

## CONCATENATE WIDGETS DATAFRAMES (GROSS SEG,CREDIT CARDS,MERCHANT,CMS,ACCOUNT OPENINGS, TD SECURITIES,BCP,WEALTH,BODP)
psa_wid_ret=pd.concat([cred_ret_filter00,cc_all_ret,merchant_all_ret,cms_all_ret,sbb_open_filter_ret,sbb_trade_filter_ret,
                     sbb_bcp_filter_ret,sbb_bodp_filter_ret,sbb_gic_filter_ret],axis=0)

##(1) group to count all metrics
amsb_cred_ret=psa_wid_ret.groupby(by=['psa_month','metric_name','am_cost_center',
                                      'am_cost_center_name','am_cost_center_full_name',
                                      'sm_cost_center','sm_cost_center_name','record_date'],as_index=False).aggregate({
                    'elig_ind': 'sum'
                     })
amsb_cred_ret.rename(columns = {"elig_ind":'count_segment'}, inplace = True)

## create Total metric for Gross Widgets (sum of Segmentation metrics)
##(a) subset the segmentation metrics
cred_sbb_filter01=psa_wid_ret[psa_wid_ret.metric_name.isin(gross_seg_tuple)]

cred_sbb_filter01['metric_name']='Total SBB Credit Widgets'

amsb_seg_ret=cred_sbb_filter01.groupby(by=['psa_month','metric_name','am_cost_center',
                                      'am_cost_center_name','am_cost_center_full_name',
                                      'sm_cost_center','sm_cost_center_name','record_date'],as_index=False).aggregate({
                    'elig_ind': 'sum'
                     })
amsb_seg_ret.rename(columns = {"elig_ind":'count_segment'}, inplace = True)

## concatenate the dataframes
amsb_all99_ret=pd.concat([amsb_cred_ret,amsb_seg_ret],axis=0)


##(2) transpose data
amsb_all90_ret=amsb_all99_ret.pivot_table(index=['metric_name','am_cost_center','am_cost_center_name',
                                        'am_cost_center_full_name','sm_cost_center','sm_cost_center_name',
                                        'record_date'], columns=['psa_month'],
                     values='count_segment', aggfunc='first').reset_index().fillna(0)


In [ ]:
################################# SUM VOLUME ####################################

## CONCATENATE VOLUME DATAFRAMES (GROSS VOLUME, COM VOLUME, MUR VOLUME)
psa_vol_ret=pd.concat([gross_all_ret,com_all_ret,sbb_mur_filter_ret],axis=0)

## group to count
psa_vol1_ret=psa_vol_ret.groupby(by=['psa_month','metric_name','am_cost_center',
                             'am_cost_center_name','am_cost_center_full_name',
                             'sm_cost_center','sm_cost_center_name','record_date'],as_index=False).aggregate({
                    'current_amount': 'sum'
                     })
psa_vol1_ret.rename(columns = {"current_amount":'vol'}, inplace = True)


##transpose data
psa_vol2_ret=psa_vol1_ret.pivot_table(index=['metric_name','am_cost_center','am_cost_center_name',
                                     'am_cost_center_full_name','sm_cost_center','sm_cost_center_name',
                                     'record_date'], columns=['psa_month'],
                     values='vol', aggfunc='first').reset_index().fillna(0)


In [ ]:
#### concatenate the widgets and volume dataframes
am_tot_ret=pd.concat([amsb_all90_ret,psa_vol2_ret],axis=0)

In [ ]:
########## CREATE ALL OF THE MONTHS
##rename columns **************CHANGE FOR FUTURE
cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)

complementary = [c for c in net_a if c not in am_tot_ret]

am_tot_ret.reset_index(drop=True, inplace=True)
psa_all_am2= pd.concat([net_a[complementary], am_tot_ret], axis=1, join='outer')

##delete the extra filler row
psa_all_am2= psa_all_am2.drop(labels='a', axis=0)
##convert to float
psa_all_am2 = psa_all_am2.astype({'november': float,'december': float,
                                'january': float, 'february': float, 'march': float,
                                'april': float, 'may': float,'june':float,'july': float,
                                'august': float, 'september':float,'october': float})

################################## group by AM ##########################################
cred_am=amgrpfunc(psa_all_am2)

##include the LEVEL variable
cred_am['level']='AM'
cred_am['scorecard_filter']='Retail'
cred_am['acf2_id']=''
cred_am['employee_name']=cred_am['am_cost_center_full_name']

##CREATE THE AM QUARTER AND YTD VARIABLES
newvarfunc(cred_am)

## replace 0 with NaN so median only counts non-zeros
cred_am=cred_am.replace(0.0, np.nan)


In [ ]:
## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
cred_am_median=cred_am.groupby(['metric_name'], as_index=False)['q1'].median()
##rename median
cred_am_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
cred_am_all3=pd.merge(cred_am,cred_am_median,on=['metric_name'],
                   how="left")

## Q2
cred_am_median1=cred_am.groupby(['metric_name'], as_index=False)['q2'].median()
##rename median
cred_am_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
cred_am_all4=pd.merge(cred_am_all3,cred_am_median1,on=['metric_name'],
                   how="left")

## Q3
cred_am_median2=cred_am.groupby(['metric_name'], as_index=False)['q3'].median()
##rename median
cred_am_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
cred_am_all5=pd.merge(cred_am_all4,cred_am_median2,on=['metric_name'],
                   how="left")

## Q4
cred_am_median3=cred_am.groupby(['metric_name'], as_index=False)['q4'].median()
##rename median
cred_am_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
cred_am_all6=pd.merge(cred_am_all5,cred_am_median3,on=['metric_name'],
                   how="left")


## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
cred_am_ytdmedian=cred_am.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
cred_am_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
cred_am_all7=pd.merge(cred_am_all6,cred_am_ytdmedian,on=['metric_name'],
                   how="left")

## keep relevant vars
cred_am_all8=cred_am_all7[['metric_name','level','acf2_id','employee_name',
               'november','december','january','february','march','april','may','june','july',
               'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
##############################  GROUP BY SM (RETAIL FILTER)##############################
cred_sm=smgrpfunc(cred_am_all8)

##include the LEVEL variable
cred_sm['level']='SM'
cred_sm['acf2_id']=''
cred_sm['scorecard_filter']='Retail'
cred_sm['employee_name']=cred_sm['sm_cost_center_name']
cred_sm['am_cost_center']=''
cred_sm['am_cost_center_name']=''
cred_sm['am_cost_center_full_name']=''

##CREATE THE AM QUARTER AND YTD VARIABLES
newvarfunc(cred_sm)

## replace 0 with NaN so median only counts non-zeros
cred_sm=cred_sm.replace(0.0, np.nan)

## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
cred_sm_median=cred_sm.groupby(['metric_name'], as_index=False)['q1'].median()
##rename median
cred_sm_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
cred_sm_all3=pd.merge(cred_sm,cred_sm_median,on=['metric_name'],
                   how="left")

## Q2
cred_sm_median1=cred_sm.groupby(['metric_name'], as_index=False)['q2'].median()
##rename median
cred_sm_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
cred_sm_all4=pd.merge(cred_sm_all3,cred_sm_median1,on=['metric_name'],
                   how="left")

## Q3
cred_sm_median2=cred_sm.groupby(['metric_name'], as_index=False)['q3'].median()
##rename median
cred_sm_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
cred_sm_all5=pd.merge(cred_sm_all4,cred_sm_median2,on=['metric_name'],
                   how="left")

## Q4
cred_sm_median3=cred_sm.groupby(['metric_name'], as_index=False)['q4'].median()
##rename median
cred_sm_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
cred_sm_all6=pd.merge(cred_sm_all5,cred_sm_median3,on=['metric_name'],
                   how="left")

## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
cred_sm_ytdmedian=cred_sm.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
cred_sm_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
cred_sm_all7=pd.merge(cred_sm_all6,cred_sm_ytdmedian,on=['metric_name'],
                   how="left")

## keep relevant vars
cred_sm_all8=cred_sm_all7[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
#### concatenate AM, SM
df_ret_all=pd.concat([cred_am_all8,cred_sm_all8],axis=0)


In [ ]:
df_ret_all.level.unique()

In [ ]:
df_ret_all.metric_name.unique()

In [ ]:
## Convert pandas dataframe to Pyspark dataframe and save in table
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
spark.createDataFrame(df_ret_all).write.mode("append").partitionBy("record_date").format("hive").saveAsTable("anp_cabbtdct1_final.SBB_SC_METRICS_RETAIL")


In [ ]:
## END OF PROGRAM